<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 15 - Customizing Model.Fit - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 15 - Customizing Model.Fit](https://www.youtube.com/watch?v=S6tLSI8bjGs&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=15)

### İçindekiler

**Loading and Preprocessing Dataset**

**Creating Model**

**Creating Custom Model (Created Train Step)**

**Creating Custom Compiler - 1**

**Creating Custom Compiler -2 (Added custom metrics)**

**Creating Custom Model (Added Test Step)**


### <font color="blue"> Giriş</font>

Bu notebookta nasıl daha esnek bir training loops oluşturabilirizi göreceğiz. Şimdiye kadar model.fit()'i kullandık ama bazen daha esnek bir yapıya ihtiyaç duyabiliyoruz. Bu notebookta nasıl customize yapacağımızı göreceğiz, diğer notebookta sıfırdan nasıl custom training loops oluşturacağımızı göreceğiz. 

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Loading and Preprocessing Dataset

In [4]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

11501568/11490434 [==============================] - 0s 0us/step


### 2. Creating Model

In [5]:
model = keras.Sequential(
    [
     layers.Input(shape=(28,28,1)),
     layers.Conv2D(64, 3, padding='same'),
     layers.ReLU(),
     layers.Conv2D(128,3, padding='same'),
     layers.ReLU(),
     layers.Flatten(),
     layers.Dense(10)
    ],
    name='model'
)

### 3. Creating Custom Model (Created Train Step)

In [6]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit,self).__init__()
    self.model = model

    # we're going to define one training step and that is going to use in a model.fit
  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.compiled_loss(y, y_pred)

    training_vars = self.trainable_variables # keras.Model bize sağlıyor bunu.
    gradients = tape.gradient(loss, training_vars) 
    # optimizer step
    self.optimizer.apply_gradients(zip(gradients, training_vars))
    
    self.compiled_metrics.update_state(y, y_pred) # this is going to be for accuracy 

    return {m.name : m.result() for m in self.metrics} 

In [ ]:
training = CustomFit(model)

* `def train_step(self, data)` fonksiyonu ile model.fit içinde çalışacak custom training stepini yazıyoruz. 

* `with tf.GradientTape() as tape:` Bunu neden yapıyoruz çünkü forward propagation yapacağız ve arkasından loss fonksiyonu uygulayacağız. Bu tape'in içinde yaptığımızda uygulanan bütün işlemler kaydedilecek bu da backpropagation için gradient hesaplamada işimize yarayacak. 

* `loss = self.compiled_loss(y, y_pred)` training.compile içerisinde loss fonksiyonu kısmında çalışacak bu kod. 

* `gradients = tape.gradient(loss, training_vars)` eğitim değişkenlerine göre lossun gradientini alıyoruz, en sonunda değiştirmek istediğimiz şey. 
* `return {m.name : m.result() for m in self.metrics}`: name-> current loss olacak. Bunu bütün metrikler için yapacağız. Bizim case'imizde bunlar loss ve accuracy.

In [7]:
training.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [8]:
training.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 498s 265ms/step - loss: 0.1081 - accuracy: 0.9676
Epoch 2/2
1875/1875 [==============================] - 495s 264ms/step - loss: 0.0412 - accuracy: 0.9875


### 4. Creating Custom Compiler - 1 

In [9]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit,self).__init__()
    self.model = model

  def compile(self, optimizer, loss): # bunu ekledik
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss

  def train_step(self, data):
    x, y = data 

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.loss(y, y_pred) # ilkinde compiled_losstu. 

    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars) 
    self.optimizer.apply_gradients(zip(gradients, training_vars))
    
    self.compiled_metrics.update_state(y, y_pred) 

    return {m.name : m.result() for m in self.metrics} 

In [ ]:
training = CustomFit(model)

In [10]:
training.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # metricsi sildik
)

* Bunu ekledik kodumuza.
```
 def compile(self, optimizer, loss):
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss
```
* `with tf.GradientTape() as tape:` içerisindeki `loss = self.compiled_loss(y, y_pred)` kodunu `loss = self.loss(y, y_pred)` ile değiştirdik.

 


* `training.compile()`'dan metricsi sildik.

In [11]:
training.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 493s 262ms/step
Epoch 2/2
1875/1875 [==============================] - 492s 262ms/step


Accuracy almadık. Kendimiz ayrıca tanımlayabiliriz bunu.

### 5. Creating Custom Compiler -2 (Added custom metrics)

In [13]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit,self).__init__()
    self.model = model

  def compile(self, optimizer, loss):
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.loss(y, y_pred) # ilkinde compiled_losstu. 

    training_vars = self.trainable_variables 
    gradients = tape.gradient(loss, training_vars) 
    self.optimizer.apply_gradients(zip(gradients, training_vars))

    acc_metric.update_state(y, y_pred) # bu değiştirildi

    return {"loss": loss, "accuracy": acc_metric.result()} # bu değiştirildi

In [14]:
training = CustomFit(model)

* `self.compiled_metrics.update_state(y, y_pred)` yerine `acc_metric.update_state(y, y_pred)` yazdık.

* `return {m.name : m.result() for m in self.metrics}` yerine `return {"loss": loss, "accuracy": acc_metric.result()}` yazdık. 

Metricsi burada tanımladık. Ama yukarıda `class CustomFit(keras.Model)` sınıfında bunu kullandık. 

In [15]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy") # bu eklendi

In [16]:
training.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [17]:
training.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 495s 264ms/step - loss: 0.0114 - accuracy: 0.9968
Epoch 2/2
1875/1875 [==============================] - 494s 264ms/step - loss: 0.0058 - accuracy: 0.9970


### 6. Creating Custom Model (Added Test Step)

`training.fit` training step üzerinde çalışıyor fakat `training.evaluate` training step üzerinde çalışıyor. Bu yüzden test_step fonksiyonu tanımlıyoruz. 

In [18]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit,self).__init__()
    self.model = model

  def compile(self, optimizer, loss):
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      loss = self.loss(y, y_pred) # ilkinde compiled_losstu. 

    training_vars = self.trainable_variables # keras.Model bize sağlıyor bunu.
    gradients = tape.gradient(loss, training_vars) 

    self.optimizer.apply_gradients(zip(gradients, training_vars))
    acc_metric.update_state(y, y_pred)

    return {"loss": loss, "accuracy": acc_metric.result()} 


  def test_step(self, data):
    x, y = data

    y_pred = self.model(x, training=False) 
    loss = self.loss(y, y_pred)
    acc_metric.update_state(y, y_pred)

    return {"loss": loss, "accuracy": acc_metric.result()} 


* `y_pred = self.model(x, training=False)` batch norm ve dropout olduğunda trainingin ve testin farklı davranması gerekiyor bu yüzden training=False ekledik.


In [19]:
training = CustomFit(model)

In [20]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")

In [21]:
training.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [22]:
training.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 495s 264ms/step - loss: 0.0074 - accuracy: 0.9983
Epoch 2/2
1875/1875 [==============================] - 496s 265ms/step - loss: 0.0048 - accuracy: 0.9981


In [23]:
training.evaluate(x_test, y_test, batch_size=32)

313/313 [==============================] - 24s 75ms/step - loss: 0.0772 - accuracy: 0.9976


[0.9973154067993164, 4.470347292340193e-08]